In [2]:
import boto3
import pandas as pd
from io import BytesIO

In [3]:
# Config cliente de S3
s3 = boto3.client('s3')

In [4]:
# Bbucket y prefijo de carpetas
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_ABTv3 = 'ABTv3/Last_releases/'

In [5]:
# Obtengo lista de carpetas en el bucket
def list_folders(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    folders = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    return folders

In [6]:
# Llamo funcion para listar carpetas
folders_v2 = list_folders(bucket_name, prefix_ABTv3)

In [7]:
# Leo los xlsx y consolido

def read_files(bucket_name, prefix):
    dfs = []
    
    # Iterar sobre cada carpeta
    for folder_name in folders_v2:
        # Obtener la lista de objetos en la carpeta
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f'{prefix}{folder_name}/')['Contents']
        
        # Buscar archivos xlsx en la carpeta
        excel_objects = [obj for obj in objects if obj['Key'].endswith('.xlsx') and '7d' not in obj['Key']]
        
        if excel_objects:
            print(excel_objects)
            # Leer el primer archivo xlsx encontrado
            obj = s3.get_object(Bucket=bucket_name, Key=excel_objects[0]['Key'])
            excel_data = obj['Body'].read() # No se puede leer con pandas directo de S3
            
            df = pd.read_excel(BytesIO(excel_data))
            # Agrego columna con el nombre de la carpeta
            df['folder_name'] = folder_name
            # Agregar el df a la lista
            dfs.append(df)
        else:
            print(f"No se encontraron archivos xlsx en la carpeta {folder_name}.")
    
    if dfs:
        # Concateno todo
        consolidated_df = pd.concat(dfs, ignore_index=True)
        return consolidated_df
    else:
        print("No se encontraron archivos xlsx en ninguna de las carpetas especificadas.")
        return None

In [9]:
# Llamo funcion para leer xlsx y consolidar en un unico DF
df = read_files(bucket_name, prefix_ABTv3)

[{'Key': 'ABTv3/Last_releases/24XORO_MEXICO/predicciones_test_2d_24XORO_MEXICO.xlsx', 'LastModified': datetime.datetime(2024, 3, 9, 4, 4, 49, tzinfo=tzlocal()), 'ETag': '"b4aad35dab2dc611e6d32d692441c0e7"', 'Size': 15891, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3/Last_releases/ABANK (TN)_EL SALVADOR/predicciones_test_2d_ABANK (TN)_EL SALVADOR.xlsx', 'LastModified': datetime.datetime(2024, 3, 9, 3, 33, 24, tzinfo=tzlocal()), 'ETag': '"c19d5fe2c3605e77905597a4a437d1ea"', 'Size': 15381, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3/Last_releases/AFEX_CHILE/predicciones_test_2d_AFEX_CHILE.xlsx', 'LastModified': datetime.datetime(2024, 3, 9, 5, 10, 50, tzinfo=tzlocal()), 'ETag': '"d4cb1885277559b3de2dff35918f9f68"', 'Size': 15453, 'StorageClass': 'STANDARD'}]
[{'Key': 'ABTv3/Last_releases/AFRO INTERNACIONAL_GUINEA/predicciones_test_2d_AFRO INTERNACIONAL_GUINEA.xlsx', 'LastModified': datetime.datetime(2024, 3, 9, 23, 35, 33, tzinfo=tzlocal()), 'ETag': '"2701d289c957bbeed0118ed803308985"', 

In [10]:
##Cortamos la base al 18/12/23##
df=df.loc[df.date<='2023-12-18']
df.date.max()

Timestamp('2023-12-18 00:00:00')

In [11]:
df.head(3)

,date,valor_real,valor_predicho,mape,error_abs,folder_name
0,2023-06-22,40879.68,36417.104002,10.916367,4462.575998,24XORO_MEXICO
1,2023-06-23,45534.13,12590.106958,72.350176,32944.023042,24XORO_MEXICO
2,2023-06-24,36969.46,58877.600103,59.260103,21908.140103,24XORO_MEXICO


In [100]:
#NECESITAMOS CALCULAR EL MAPE PROMEDIO DIARIO A DOS DIAS Y EL ERROR ABSOLUTO DIARIO A 2D##
## POR PAYER Y POR COUNTRY##

In [101]:
## SE CORRIGEN LOS VALORES PREDICHOS NEGATIVOS##

In [12]:
valor_a_reemplazar = 0
df['valor_predicho'] = df['valor_predicho'].apply(lambda x: valor_a_reemplazar if x < 0 else x)

In [103]:
#df.to_excel('predicciones_all_payers_2d.xlsx')

In [20]:
import numpy as np
def mean_without_inf(x):
    x_filtered = x.replace([np.inf, -np.inf], np.nan).dropna()
    return np.mean(x_filtered)

# Aplicar la función personalizada en el método agg
df_ABTv3 = df.groupby('folder_name').agg({'mape': mean_without_inf, 'error_abs': 'mean', 'valor_real': 'sum'}).reset_index()


In [21]:
df_ABTv3.rename(columns={'folder_name': 'payer_country', 'mape': 'mape_v3', 'error_abs': 'error_abs_mean', 'valor_real': 'sum_amount'}, inplace=True)

In [22]:
df_ABTv3=df_ABTv3.sort_values(by='mape_v3')

In [24]:
#df_ABTv3.to_excel('mapes_prom_diarios_f_2d_14_03_24.xlsx')

In [108]:
##AHORA NECESITO HACER LO MISMO PERO CALCULANDO EL ERROR A UNA SEMANA (SIENDO SEMANA LA UNIDAD DE MEDIDA)
##CUIDADO QUE SON LOS EXCEL DE MODELO CON PREDICT A 2 D DE MANERA RECURSIVA PERO ACÁ LOS AGRUPA SEMANALMENTE
## TOMAR EL MODELO FINAL DE 7D
## NO RELEVANTE ESTE CALCULO EN EL MODELO A 2 D

In [25]:
df.head(3)

,date,valor_real,valor_predicho,mape,error_abs,folder_name
0,2023-06-22,40879.68,36417.104002,10.916367,4462.575998,24XORO_MEXICO
1,2023-06-23,45534.13,12590.106958,72.350176,32944.023042,24XORO_MEXICO
2,2023-06-24,36969.46,58877.600103,59.260103,21908.140103,24XORO_MEXICO


In [26]:
df.index = df.date

In [27]:
#df.index = pd.to_datetime(df.index)
df['semana_del_año'] = df.index.to_series().dt.isocalendar().week


In [112]:
df.head()

,date,valor_real,valor_predicho,mape,error_abs,folder_name,semana_del_año
date,,,,,,,
2023-06-22,2023-06-22,40879.68,36417.104002,10.916367,4462.575998,24XORO_MEXICO,25
2023-06-23,2023-06-23,45534.13,12590.106958,72.350176,32944.023042,24XORO_MEXICO,25
2023-06-24,2023-06-24,36969.46,58877.600103,59.260103,21908.140103,24XORO_MEXICO,25
2023-06-25,2023-06-25,59696.11,88875.091459,48.879201,29178.981459,24XORO_MEXICO,25
2023-06-26,2023-06-26,36271.54,30573.658875,15.708958,5697.881125,24XORO_MEXICO,26


In [28]:
df2=df.copy()
df2=df.loc[:,['folder_name', 'valor_real', 'valor_predicho', 'semana_del_año']]

In [114]:
df_semana = df2.groupby(['semana_del_año', 'folder_name']).sum()

In [115]:
df_semana['error_abs_7d']=abs(df_semana.valor_real - df_semana.valor_predicho)

In [116]:
df_semana['mape_7d']=df_semana.error_abs_7d / df_semana.valor_real

In [117]:
df_semana.head()

valor_real  valor_predicho  \
semana_del_año folder_name                                                   
25             24XORO_MEXICO                     183079.38   196759.902522   
               ABANK (TN)_EL SALVADOR             71151.00    70322.767421   
               AFEX_CHILE                         11585.46     9910.106515   
               AFRO INTERNACIONAL_GUINEA           1200.00      389.918786   
               AFRO INTERNACIONAL_SIERRA LEONE    31254.46    53288.724939   

                                                error_abs_7d   mape_7d  
semana_del_año folder_name                                              
25             24XORO_MEXICO                    13680.522522  0.074725  
               ABANK (TN)_EL SALVADOR             828.232579  0.011640  
               AFEX_CHILE                        1675.353485  0.144608  
               AFRO INTERNACIONAL_GUINEA          810.081214  0.675068  
               AFRO INTERNACIONAL_SIERRA LEONE  22034.264939  0.704996

In [118]:
#df_semana.to_excel('forecast_semanal_14_03_2024.xlsx')

In [119]:
##Misma funcion que la aplicada al caso diario, para no contemplar en la media de mapes los casos inf##
def mean_without_inf(x):
    x_filtered = x.replace([np.inf, -np.inf], np.nan).dropna()
    return np.mean(x_filtered)

# Aplicar la función personalizada en el método agg
df_ABTv3_7d = df_semana.groupby('folder_name').agg({'mape_7d': mean_without_inf , 'error_abs_7d': 'mean'}).reset_index()

In [120]:
df_ABTv3_7d=df_ABTv3_7d.sort_values(by='mape_7d')

In [121]:
#df_ABTv3_7d.to_excel('mapes_prom_semanal_f_7d_14_03_2024.xlsx')

#### By Country

In [29]:
df['country'] = df['folder_name'].apply(lambda x: x.split('_')[-1])
df['date']=pd.to_datetime(df['date'])
df.reset_index(drop=True, inplace=True)

In [30]:
df.head()

,date,valor_real,valor_predicho,mape,error_abs,folder_name,semana_del_año,country
0,2023-06-22,40879.68,36417.104002,10.916367,4462.575998,24XORO_MEXICO,25,MEXICO
1,2023-06-23,45534.13,12590.106958,72.350176,32944.023042,24XORO_MEXICO,25,MEXICO
2,2023-06-24,36969.46,58877.600103,59.260103,21908.140103,24XORO_MEXICO,25,MEXICO
3,2023-06-25,59696.11,88875.091459,48.879201,29178.981459,24XORO_MEXICO,25,MEXICO
4,2023-06-26,36271.54,30573.658875,15.708958,5697.881125,24XORO_MEXICO,26,MEXICO


In [32]:
df_country=df.groupby(['country','date']).agg({'valor_real': 'sum', 'valor_predicho': 'sum', 'valor_real': 'sum'}).reset_index()

In [33]:
###DESVIO ABSOLUTO POR DIA##
df_country['error_abs']= abs(df_country.valor_predicho - df_country.valor_real)

###MAPE DIA COUNTRY##
df_country['mape']= df_country.error_abs/df_country.valor_real

In [42]:
promedio_mape_por_country = df_country.groupby('country').agg({'mape': mean_without_inf , 'error_abs': 'mean', 'valor_real': 'sum' }).reset_index()

In [43]:
promedio_mape_por_country.rename(columns={'valor_real': 'sum_amount'}, inplace=True)

In [44]:
promedio_mape_por_country['mape']=promedio_mape_por_country.mape * 100

In [45]:
promedio_mape_por_country.sort_values(by='mape', ascending=True)

,country,mape,error_abs,sum_amount
14,EL SALVADOR,6.500173,9.142135e+04,2.599624e+08
22,HONDURAS,7.066390,1.410554e+05,3.663244e+08
30,MEXICO,7.117630,1.242966e+06,3.367170e+09
19,GUATEMALA,7.118080,5.564267e+05,1.469552e+09
12,DOMINICAN REPUBLIC,9.774677,4.014705e+04,7.680630e+07
32,NICARAGUA,10.737507,7.215015e+04,1.346826e+08
13,ECUADOR,11.156990,5.925907e+04,9.895877e+07
34,PERU,13.268858,1.929266e+04,2.841291e+07
9,COLOMBIA,13.931821,9.201758e+04,1.325347e+08
21,HAITI,14.640847,6.264788e+03,8.560913e+06


In [46]:
promedio_mape_por_country.shape

(44, 4)

In [ ]:
##HASTA ACÁ CON LA SALIDA DE LOS MODELOS DE FORECAST EN BACKTESTING A 2D##

In [47]:
promedio_mape_por_country=promedio_mape_por_country.sort_values(by='mape', ascending=True)
promedio_mape_por_country.to_excel('mapes_promedio_por_country_14_03_2024_f_2d.xlsx')